In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
from pydub import AudioSegment

# Dossier contenant les vidéos téléchargées
songs_folder = "./top_songs_database"

# Dossier pour enregistrer les spectrogrammes
spectrogram_folder = "spectrogrammes"
if not os.path.exists(spectrogram_folder):
    os.makedirs(spectrogram_folder)

# Fonction pour générer le spectrogramme à partir de l'audio
def generate_spectrogram(audio_path, title):
    # Charger l'audio avec librosa
    y, sr = librosa.load(audio_path, sr=22050)
    spectrogram = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

    # Créer le spectrogramme
    plt.figure(figsize=(12, 6))
    librosa.display.specshow(spectrogram, sr=sr, x_axis="time", y_axis="log", cmap="magma")
    plt.colorbar(format="%+2.0f dB")
    plt.title(f"Spectrogramme de {title}")

    # Enregistrer le spectrogramme
    spectrogram_path = os.path.join(spectrogram_folder, f"{title}.png")
    plt.savefig(spectrogram_path)
    plt.close()

# Parcours du dossier "top_songs_database" et traitement de chaque fichier vidéo/audio
for filename in os.listdir(songs_folder):
    song_path = os.path.join(songs_folder, filename)

    if filename.endswith(('.mp3', '.wav')):  # Si le fichier est un audio
        audio_path = song_path
        title = os.path.splitext(filename)[0]
        print(f"Traitement de l'audio: {title}")
        generate_spectrogram(audio_path, title)

    elif filename.endswith(('.mp4', '.mkv', '.webm')):  # Si le fichier est une vidéo
        audio_path = "audio_from_video.wav"
        title = os.path.splitext(filename)[0]
        
        # Extraire l'audio de la vidéo
        print(f"Extraction de l'audio de la vidéo: {title}")
        audio = AudioSegment.from_file(song_path)
        audio.export(audio_path, format="wav")

        # Générer le spectrogramme pour l'audio extrait
        generate_spectrogram(audio_path, title)

        # Supprimer le fichier audio temporaire
        os.remove(audio_path)

print(f"Tous les spectrogrammes ont été générés et sauvegardés dans le dossier '{spectrogram_folder}'")
